In [1]:
import traitlets
import ipywidgets.widgets as widgets
from ipywidgets import HTML
from IPython.display import display
import cv2
import os
import glob

import PIL.Image
from PIL import Image
from matplotlib import pyplot as plt

In [2]:
img_path1 = 'sample.png'
class_path = 'classes/'

classes = ['left_near', 'left_just', 'left_far', 
            'middle_near', 'middle_just', 'middle_far', 
            'right_near', 'right_just', 'right_far']

for s in classes:
    os.makedirs("classes/"+s, exist_ok=True)

In [20]:
def fx(x1, x2, x, img): # 座標(x1, 0),(x2, img.shape[0])の2点を通る直線の式における、xがx(引数)の時のyを返す
    y = img.shape[0] // (x2 - x1) * (x - x1)
    return y
    
def directionClassify(x1, x2, x, y, img_path): # 方向分類
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224)) 
    
    img_width = 224
    
    
    x1 = x1 * img.shape[1] // img_width
    x2 = x2 * img.shape[1] // img_width
    
    mid_x = img.shape[1] // 2
    new_x1 = 2 * mid_x - x1
    new_x2 = 2 * mid_x - x2
    
    if x1 < x2:
        if (x == x1 and y == 0) or (x == new_x1 and y == 0) or (x >= x2 and x <= new_x2):
            direction = 'middle'
        elif x > x1 and x < x2 and y <= fx(x1, x2, x, img):
            direction = 'middle'
        elif x > new_x2 and x < new_x1 and y <= fx(new_x1, new_x2, x, img):
            direction = 'middle'
        elif x >= 0 and x <= x1:
            direction = 'left'
        elif x > x1 and x < x2 and y > fx(x1, x2, x, img):
            direction = 'left'
        elif x >= new_x1 and x <= img.shape[1]:
            direction = 'right'
        elif x > new_x2 and x < new_x1 and y > fx(new_x1, new_x2, x, img):
            direction = 'right'
    elif x1 > x2:
        if (x == x2 and y == img.shape[0]) or (x == new_x2 and y == img.shape[0]) or (x >= x1 and x <= new_x1):
            direction = 'middle'
        elif x > x2 and x < x1 and y >= fx(x1, x2, x, img):
            direction = 'middle'
        elif x > new_x1 and x < new_x2 and y >= fx(new_x1, new_x2, x, img):
            direction = 'middle'
        elif x >= 0 and x <= x2:
            direction = 'left'
        elif x > x2 and x < x1 and y < fx(x1, x2, x, img):
            direction = 'left'
        elif x >= new_x2 and x <= img.shape[1]:
            direction = 'right'
        elif x > new_x1 and x < new_x2 and y < fx(new_x1, new_x2, x, img):
            direction = 'right'
    elif x1 == x2:
        if x >= 0 and x < x1:
            direction = 'left'
        elif x > new_x1 and x <= img.shape[1]:
            direction = 'right'
        else:
            direction = 'middle'
            
    return direction
        
def distanceClassify(rad_start, rad_end, rad, img_path): # 距離分類
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224)) 
    img_width = 224
    
    
    #start = img.shape[1] // rad_start // 2 # justの下限
    #end = img.shape[1] // rad_end // 2 # justの上限
    
    start = rad_start * img.shape[1] // img_width
    end = rad_end * img.shape[1] // img_width
    
    if rad > 0 and rad < start:
        distance = 'far'
    elif rad > end:
        distance = 'near'
    else:
        distance = 'just'
        
    return distance
    
def allClassify(x1, x2, x, y, rad_start, rad_end, rad, img_path): # 全体の分類
    
    """ 修正　"""
    
    img = cv2.imread(img_path)
    #original_img = img
    output_folder = ''
    
    direction = directionClassify(x1, x2, x, y, img_path)
    distance = distanceClassify(rad_start, rad_end, rad, img_path)
      
        
        
    global file_name
    output_folder = class_path+direction+'_'+distance+'/' + file_name
    original_img = Image.open(img_path)
    original_img.save(output_folder)
        
    print(output_folder)
    boundaryTest(x1, x2, x, y, rad_start, rad_end, rad, img_path)
    
def boundaryTest(x1, x2, x, y, rad_start, rad_end, rad, img_path): # しきい値の確認
    img = cv2.imread(img_path)
    img = cv2.resize(img,(224,224)) 
    img_width = 224
    
    
    x1 = x1 * img.shape[1] // img_width
    x2 = x2 * img.shape[1] // img_width
    mid_x = img.shape[1] // 2
    new_x1 = 2 * mid_x - x1
    new_x2 = 2 * mid_x - x2
    
    #start = img.shape[1] // rad_start // 2
    #end = img.shape[1] // rad_end // 2
    
    start = rad_start * img.shape[1] // img_width
    end = rad_end * img.shape[1] // img_width

    ##可視化
#     red_color = (0, 0, 255)
#     green_color = (0, 255, 0)
#     blue_color = (255, 0, 0)
#     yellow_color = (0, 255, 255)
#     thickness = 2
#     circle_thickness = -1
    
#     cv2.circle(img, (x, y), rad, yellow_color, thickness)
#     cv2.line(img, (x1, 0), (x2, img.shape[0]), blue_color, thickness)
#     cv2.line(img, (new_x1, 0), (new_x2, img.shape[0]), blue_color, thickness)
#     cv2.circle(img, (x, y), start, green_color, thickness)
#     cv2.circle(img, (x, y), end, green_color, thickness)
#     cv2.circle(img, (x, y), 2, red_color, circle_thickness)

    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.imshow(img)
    plt.show()


""" 追加　"""

def getrate(img_path):
    ###Resizeする倍率を取得 座標や半径の調整に利用###
    img = cv2.imread(img_path)
    
    return img.shape[0] / 224,img.shape[1] / 224
    

def get_x(path):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[1])) ) 

def get_y(path):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[2])) ) 

def get_r(path):
    """Gets the y value from the image filename"""
    return (float(int(path.split("_")[3])) )

In [21]:
# インタラクティブなウィジェットを作成
x1_slider = widgets.IntSlider(min=0, max=224/2, value=10, description='x1:')
x2_slider = widgets.IntSlider(min=0, max=224/2, value=10, description='x2:')

x_slider = widgets.IntSlider(min=0, max=224, value=112, description='X:')
y_slider = widgets.IntSlider(min=0, max=224, value=112, description='Y:')
rad_start_slider = widgets.IntSlider(min=0, max=300, value=50, description='Rad Start:')
rad_end_slider = widgets.IntSlider(min=0, max=300, value=100, description='Rad End:')

img_widget_1 = widgets.Image(format='png', width=224, height=224)
img_widget_2 = widgets.Image(format='png', width=224, height=224)

In [22]:
def LineTest(change):
    global img_path1
    image = cv2.imread(img_path1)
    

    x1 = x1_slider.value
    x2 = x2_slider.value
    mid_x = 112
    mid_y = 112
    new_x1 = 2 * mid_x - x1
    new_x2 = 2 * mid_x - x2
    
    
    blue_color = (255, 0, 0)
    thickness = 2
    
    img = cv2.line(image, (int(x1), 0), (int(x2), image.shape[0]), blue_color, thickness)
    img = cv2.line(image, (new_x1, 0), (new_x2, image.shape[0]), blue_color, thickness)
    
    
    _, encoded_image = cv2.imencode('.png', img)
    img_widget_1.value = encoded_image.tobytes()
        
    


    
def CircleTest(change):
    global img_path1
    img = cv2.imread(img_path1)
    
    x = x_slider.value
    y = y_slider.value
    rad_start = rad_start_slider.value
    rad_end = rad_end_slider.value

    # 色と太さの設定
    red_color = (0, 0, 255)
    green_color = (0, 255, 0)
    yellow_color = (0, 255, 255)
    blue_color = (255 , 0 , 0)
    circle_thickness = -1
    thickness = 2

    # 円を描画
    img = cv2.circle(img, (x, y), rad_start, blue_color, thickness)
    img = cv2.circle(img, (x, y), rad_end, green_color, thickness)
    img = cv2.circle(img, (x, y), 2, red_color, circle_thickness)

    _, encoded_image = cv2.imencode('.png', img)
    img_widget_2.value = encoded_image.tobytes()
    

LineTest(img_path1)
CircleTest(img_path1)

In [23]:
# スライダーの値が変更されたときにコールバック関数を実行
x1_slider.observe(LineTest, names='value')
x2_slider.observe(LineTest, names='value')

# スライダーの変更を監視
x_slider.observe(CircleTest, names='value')
y_slider.observe(CircleTest, names='value')
rad_start_slider.observe(CircleTest, names='value')
rad_end_slider.observe(CircleTest, names='value')

In [24]:
class_html=HTML(
    value="クラス分類"
)
# 水平線を引くための HTML ウィジェットを作成
horizontal_line = widgets.HTML(
    value="<hr style='border-top: 1px solid black;'>",
    layout=widgets.Layout(width='100%')
)

# 空白を作成
def space(width,height):
    width = str(width)+'px'
    height = str(height)+'px'
    return widgets.Box(layout=widgets.Layout(width = width,height = height))



#####境界線
image_layout_2_1 = widgets.HBox([img_widget_1])
xy_option_layout = widgets.VBox([x1_slider,x2_slider]) 
line_layout = widgets.VBox([image_layout_2_1,xy_option_layout])
gui_layout_2_1 = widgets.HBox([line_layout])

#####境界円
image_layout_2_2 = widgets.HBox([img_widget_2])
xy_option_layout_2 = widgets.VBox([x_slider,y_slider,rad_start_slider,rad_end_slider])
circle_layout = widgets.VBox([image_layout_2_2,xy_option_layout_2])
gui_layout_2_2 = widgets.HBox([circle_layout])

In [25]:
classificate_layout = widgets.HBox([gui_layout_2_1,gui_layout_2_2])
classificate_layout = widgets.VBox([horizontal_line,class_html,classificate_layout,space(0,30)])


In [26]:
# display(classificate_layout)

In [27]:
#data_path = 'dataset/'
data_path = '../generate/image/paste/'
class_path = 'classes/'
classes = ['left_near', 'left_just', 'left_far', 
            'middle_near', 'middle_just', 'middle_far', 
            'right_near', 'right_just', 'right_far']

button_layout = widgets.Layout(width='128px', height='64px')
counters = [widgets.IntText(layout=button_layout, value=len(os.listdir(class_path+classes[0]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir(class_path+classes[1]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir(class_path+classes[2]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir(class_path+classes[3]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir(class_path+classes[4]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir(class_path+classes[5]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir(class_path+classes[6]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir(class_path+classes[7]))),
           widgets.IntText(layout=button_layout, value=len(os.listdir(class_path+classes[8])))]

def create_html(num):
    
    global classes
    return HTML(
        value= classes[num]   
        )

data_button = widgets.Button(description='run', button_style='success', layout=button_layout)
reload_button = widgets.Button(description='load', button_style='success', layout=button_layout)
delete_button = widgets.Button(description='delete', button_style='danger', layout=button_layout)


left_layout = widgets.VBox([create_html(0),counters[0],create_html(1),counters[1],create_html(2),counters[2]])
middle_layout = widgets.VBox([create_html(3),counters[3],create_html(4),counters[4],create_html(5),counters[5]])
right_layout = widgets.VBox([create_html(6),counters[6],create_html(7),counters[7],create_html(8),counters[8]])

save_count = widgets.IntText(layout=button_layout, value=len(glob.glob(os.path.join(data_path, '*.jpg'))))
button_layout_2 = widgets.VBox([space(0,0),data_button,space(0,30),reload_button,save_count,space(0,30),delete_button])



all_layout = widgets.HBox([button_layout_2,space(30,0),left_layout,space(30,0),middle_layout,space(30,0),right_layout])


In [28]:
file_name = ''


def reload():
    classes = ['left_near', 'left_just', 'left_far', 
            'middle_near', 'middle_just', 'middle_far', 
            'right_near', 'right_just', 'right_far']
    
    
    for i in range(len(classes)):  
        
        counters[i].value = len(glob.glob(os.path.join(class_path+classes[i], '*.jpg')))+len(glob.glob(os.path.join(class_path+classes[i], '*.png')))
        
    save_count.value = len(glob.glob(os.path.join(data_path, '*.jpg'))) + len(glob.glob(os.path.join(data_path, '*.png')))
    

def run_classificate():
    global data_path
    entries = os.listdir(data_path)
    # Filter to include only .jpg files
    img_files = [file for file in entries if file.endswith(('.jpg','.png'))]

    global file_name
    
    for img in img_files:
        
        file_name = img
        img_path = data_path+file_name
        
        kx,ky = getrate(img_path)
        x= int(get_x(file_name) / kx)
        y = int(get_y(file_name) / ky)
        rad = int(get_r(file_name) / kx)
        
        allClassify(int(x1_slider.value), int(x2_slider.value), x, y, int(rad_start_slider.value), int(rad_end_slider.value), rad, img_path)

        
def delete_files_in_folder(directory):
    try:
        for filename in os.listdir(directory):
            file_path = os.path.join(directory, filename)
            # ファイルのみを対象とし、ディレクトリは無視
            if os.path.isfile(file_path):
                os.remove(file_path)
                print(f"Deleted file: {file_path}")
            else:
                print(f"Skipped directory: {file_path}")
    except Exception as e:
        print(f"An error occurred: {e}")

def delete(class_path = class_path):
    
    classes = ['left_near', 'left_just', 'left_far', 
            'middle_near', 'middle_just', 'middle_far', 
            'right_near', 'right_just', 'right_far']
    
    for s in classes:
        delete_files_in_folder(class_path+s)
    
        
data_button.on_click(lambda x: run_classificate())
reload_button.on_click(lambda x: reload())
delete_button.on_click(lambda x:delete())

In [29]:
# display(all_layout)

In [30]:

display(classificate_layout)
display(all_layout)